In [1]:
import tensorflow as tf
import numpy as np

In [29]:
tf.reset_default_graph()
sentences = ["i like dog", "i love coffee", "i hate milk"]

n_hidden = 5
n_step = 2

word_dict = ' '.join(sentences).split()
word_dict = list(set(word_dict))
word_num = {w:i for i,w in enumerate(word_dict)}
num_word = {i:w for i,w in enumerate(word_dict)}
n_class = len(word_num)

def get_batch(sentences):
    input_batch = []
    target_batch = []
  
    for sen in sentences:
        word = sen.split()
        input_temp = [word_num[w] for w in word[:-1]]
        target_temp = word_num[word[-1]]
        
        input_batch.append(np.eye(n_class)[input_temp])
        target_batch.append(np.eye(n_class)[target_temp])
        
    return input_batch,target_batch

X = tf.placeholder(tf.float32,[None,n_step,n_class])
Y = tf.placeholder(tf.float32,[None,n_class])

W = tf.Variable(tf.random_normal([n_hidden,n_class]))
b = tf.random_normal([n_class])

#model
cell = tf.contrib.rnn.BasicRNNCell(n_hidden)
cell = tf.contrib.rnn.DropoutWrapper(cell,output_keep_prob=0.5)
hidden_outputs,last_outputs = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)# [n_step, batch_size, n_hidden]
hidden_outputs = tf.transpose(hidden_outputs,[1,0,2]) #[n_step, batch_size, n_hidden]
outputs = hidden_outputs[-1]#[batch_size, n_hidden]

out = tf.matmul(outputs,W) + b #[batch_size,n_class]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=out))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

predict = tf.cast(tf.argmax(out,1),tf.int32)

input_batch,target_batch = get_batch(sentences)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
        _,loss = sess.run([optimizer,cost],feed_dict={X:input_batch,Y:target_batch})
        
        if (i+1)%1000 == 0:
            print('epoch:%d loss:%.6f' % ((i+1),loss))
        
    test = ["i like dog"]
    test_x,_ = get_batch(test)
    pre = sess.run(predict,feed_dict={X:test_x})
    print(num_word[pre[0]])

epoch:1000 loss:0.650059
epoch:2000 loss:0.008774
epoch:3000 loss:0.086917
epoch:4000 loss:0.983347
epoch:5000 loss:0.093300
dog
